# Introduction
During my orchestra [LiTHe Blås](http://litheblas.org)'s 45 year aniversary I will be doing a customary "Skitsnack" between two songs. 
A "Skitsnack" is where an orchestra member keeps the audience preoccupied by talking about anything between heaven and earth until we start playing the next song.
I often find it hard to decide what I should talk about and usually end up telling really bad jokes or rambling about some algorithm I just read about. 

For this "Skitsnack" I will avoid coming up with my own material entierly by generating it with an n-grams model! 
The focus of this Notebook is simply to generate a short text which I think could entertain a group of 500 for about 30-60 seconds.

There are some nice aspects of the task of generating a "funny" text from a predefined dictionary:
* I can easily perform extrinsic testing. "Is this text funny?"
* I don't have to worry about unkown words.

As such, any quantitative evaluation will only happen in the spur of the moment.

Inspiration:
https://web.stanford.edu/~jurafsky/slp3/4.pdf

# Training data
I haven't decided what data I should use to train my model, and will probably have to try some different sources when the model is finished. The final text will definitely be in Swedish to accomodate the audience, but to get started I will use the Dansih author [H.C. Andersen's Fairy Tales](http://www.gutenberg.org/ebooks/32572) translated to English.

## Cleaning the data

In [1]:
import codecs

In [2]:
with codecs.open('hcandersen_fairy_tales.txt', 'r', encoding='utf-8') as f:
    text = f.read()

First let's remove all text which is not part of his stories, as I don't want to use this for training.

In [3]:
import re

In [4]:
for i in re.finditer(r"HANS ANDERSEN'S FAIRY TALES", text):
    print(i.start(0), i.end(0))
    # Hiding the output for future readability
    #print(text[i.start(0): i.end(0) + 200])
    #print("###########################")

627 654
4021 4048
4140 4167
375029 375056


In [5]:
text = text[4167:]

In [6]:
for i in re.finditer(r"NOTES", text):
    print(i.start(0), i.end(0))
    print(text[i.start(0): i.end(0) + 200])
    print("###########################")

367113 367118
NOTES


THE STORKS

          PAGE 29. On account of the ravages it makes among
          noxious animals, the stork is a privileged bird
          wherever it makes its home. In cities it is
     
###########################


In [7]:
text = text[:367113 ]

Let's remove all tabs and linebreaks.

In [8]:
text = re.sub(r'\r', '', text)
text = re.sub(r'\n', ' ', text)

## Tokenization
### Sentences
First let's parse the text as sentences using NLTK.

In [9]:
from nltk.tokenize import sent_tokenize

In [10]:
sentences = sent_tokenize(text)

In [11]:
print(sentences[0])
print(sentences[1])

     THE FLAX   THE flax was in full bloom; it had pretty little blue flowers, as delicate as the wings of a moth.
The sun shone on it and the showers watered it; and this was as good for the flax as it is for little children to be washed and then kissed by their mothers.


The first one is not good, it includes the story title. I will noth bother with this at the moment though, as this is not the text I will be using for my final results.

### Words

In [12]:
from nltk.tokenize import word_tokenize

In [13]:
sentences = list(map(word_tokenize, sentences))

In [14]:
print(sentences[1])

['The', 'sun', 'shone', 'on', 'it', 'and', 'the', 'showers', 'watered', 'it', ';', 'and', 'this', 'was', 'as', 'good', 'for', 'the', 'flax', 'as', 'it', 'is', 'for', 'little', 'children', 'to', 'be', 'washed', 'and', 'then', 'kissed', 'by', 'their', 'mothers', '.']


### Add beginning and end of sentence tags
All my sentences need a root and a stop token. I will insert a `<BOS>` tag before the first word of each sentence and replace the last punctuation of each sentence with `<EOS>`, I do this to distinguish between in sentence punctuation and the actual end of sentence. 

In [15]:
sentences = list(map(lambda x: ['<BOS>'] + x[:-1] + ['<EOS>'], sentences))

In [16]:
print(sentences[1])

['<BOS>', 'The', 'sun', 'shone', 'on', 'it', 'and', 'the', 'showers', 'watered', 'it', ';', 'and', 'this', 'was', 'as', 'good', 'for', 'the', 'flax', 'as', 'it', 'is', 'for', 'little', 'children', 'to', 'be', 'washed', 'and', 'then', 'kissed', 'by', 'their', 'mothers', '<EOS>']


# N-Gram model

## Constructing n-grams from the setnences
Let's try the nltk ngrams package

In [17]:
from nltk import ngrams

## Implementing a generative model
When generating my text I will not be considering context across sentences. Each `<EOS>` tag will be followed by a fresh `<BOS>`, interpreted as a unigram.

In [18]:
from functools import reduce
import numpy as np
import pandas as pd

In [117]:
class NGrams():
    def __init__(self, n):
        self.n = n
        
    def fit(self, sentences):
        # Count all ngrams
        self.grams = []
        for i in range(1, self.n+1):
            # Build a sorted list of all ngrams
            grams = list(map(lambda x: list(ngrams(x, i)), sentences))
            grams = np.array(reduce(lambda x, y: x + y, grams))
            grams = grams[np.lexsort(grams[:,::-1].T,),:]
            
            # Build an array marking the first unique occurence of a n-gram
            # Example unigrams, 
            # a = [('a'), ('a'), ('b'), ('c'), ('c')]
            # -> [True, False, True, True, False]
            first_occurence = np.append([True], np.array([np.any(grams[i-1] != grams[i]) for i in range(1, len(grams))]))
            
            # Assign each unique n-gram an index
            # Example unigrams, 
            # a = [('a'), ('a'), ('b'), ('c'), ('c')]
            # -> [0, 0, 1, 2, 2]
            ids = np.append([0], first_occurence[1:].cumsum())
            
            # Build a mapping from n-gram to count
            frequencies = dict(list(zip(map(tuple, grams[first_occurence,:]), np.bincount(ids))))
            self.grams.append(frequencies)
        
        # Record the total number of unigrams
        self.n_unigrams = sum(self.grams[0].values())         
        
    def log_prob(self, word, context):
        n = len(context)
        gram = context + (word,)
        if n == 0:
            return np.log(self.grams[n][gram] / self.n_unigrams)
        else:
            return np.log(self.grams[n][gram] / self.grams[n-1][context])      

In [97]:
len(sentences)

3902

In [105]:
trigram = NGrams(3)

In [106]:
%time trigram.fit(sentences)

Wall time: 7.99 s


In [23]:
trigram.log_prob('the', ('!', '--'))

0.0

In [24]:
trigram.log_prob('call', ("'ll",))

-3.4657359027997265

In [25]:
trigram.log_prob('call', ())

-8.7714179106038408

Cool, we can calculate the log probabilities of a word following an ngram. To avoid overfitting to the exact sentences in the text some smoothing should be done. Without smoothing we will assign zero probability to all ngrams that are not present in the text.

However, I have encountered a problem: At the moment I don't feel creative enough to come up with a generative model that finds the next word in a sentence based on smoothed probabilities.

I will start of with an unsmoothed model, and at the same time also switch to a tree structure to represent the model vocabulary.

## Simple text generating model

In [131]:
import random
import bisect

In [248]:
class NGramsTree():
    def __init__(self, n):
        self.n = n
    def fit(self, sentences):
         # Build a sorted list of all ngrams
        grams = list(map(lambda x: list(ngrams(x, self.n)), sentences))
        grams = np.array(reduce(lambda x, y: x + y, grams))
        
        # Construct a tree based on the ngrams
        self.tree = {}
        self.tree['count'] = len(grams)
        self.tree['words'] = {}
        for gram in grams:
            reference = self.tree
            for i, word in enumerate(gram):
                if word in reference['words']: 
                    reference = reference['words'][word]
                    reference['count'] += 1
                else:
                    reference['words'][word] = {}
                    reference = reference['words'][word]
                    reference['count'] = 1
                    # Don't create maps for the leaves
                    if i != self.n - 1:
                        reference['words'] = {}
                        
        # Create integer bins for all words in each context based on their apperances
        # These bins will be used to find a random successor to contexts with probabilities
        # relative to number of appearences. 
        # Note: Keys in a dictionary are stored in a non-deterministic, but stable, order.
        # Therefore associating the bin with index 0 with the key at index 0 is OK.
        def build_bins(tree):
            if 'words' not in tree:
                return
            else:
                tree['bins'] = [0]
                for values in tree['words'].values():
                    tree['bins'].append(tree['bins'][-1] + values['count'])
                    build_bins(values)
        
        build_bins(self.tree)

        
    def generate_sentence(self, sentence, max_length = 100):
        n = min(len(sentence), self.n - 1)
        reference = self.tree
        #print(sentence, sentence[-n:])
        for word in sentence[-n:]:
            reference = reference['words'][word]
        rand_i = random.randint(0, reference['count'] - 1)
        word_index = bisect.bisect_right(reference['bins'], rand_i) - 1
        word = list(reference['words'].keys())[word_index]
        sentence.append(word)
        if word == "<EOS>" or len(sentence) == max_length:
            return sentence
        else:
            return self.generate_sentence(sentence)
         
        

In [278]:
treegram = NGramsTree(3)

In [279]:
example_sentences = [
    ['a', 'a', 'a'],
    ['a', 'a', 'a'],
    ['a', 'a', 'c'],
    ['a', 'b', 'a'],
    ['b', 'a', 'a'],
]

In [280]:
%time treegram.fit(example_sentences)

Wall time: 516 µs


In [281]:
import json

Here is an example of what the tree structure looks like for trigrams.

In [283]:
print(json.dumps(treegram.tree, sort_keys=True,
                      indent=4, separators=(',', ': ')))

{
    "bins": [
        0,
        4,
        5
    ],
    "count": 5,
    "words": {
        "a": {
            "bins": [
                0,
                3,
                4
            ],
            "count": 4,
            "words": {
                "a": {
                    "bins": [
                        0,
                        2,
                        3
                    ],
                    "count": 3,
                    "words": {
                        "a": {
                            "count": 2
                        },
                        "c": {
                            "count": 1
                        }
                    }
                },
                "b": {
                    "bins": [
                        0,
                        1
                    ],
                    "count": 1,
                    "words": {
                        "a": {
                            "count": 1
                        }
                  

In [292]:
treegram = NGramsTree(4)
treegram.fit(sentences)

In [293]:
for i in range(5):
    print(treegram.generate_sentence(sentence = ['<BOS>']))

['<BOS>', 'He', 'divided', 'his', 'bread', 'and', 'water', 'with', 'me', 'and', 'gave', 'me', 'cheese', 'and', 'sausage', ',', 'and', 'I', 'told', 'her', 'I', 'felt', 'myself', 'quite', 'innocent', 'of', 'any', 'such', 'pretensions', '<EOS>']
['<BOS>', '``', 'Yes', ',', 'but', 'with', 'something', 'in', 'it', 'to', 'laugh', 'at', ',', "''", 'said', 'he', ';', '``', 'there', 'is', 'no', 'help', 'for', 'it', ';', 'you', 'shall', 'have', 'your', 'way', ',', 'though', 'it', 'will', 'bring', 'you', 'to', 'sorrow', ',', 'my', 'pretty', 'princess', '<EOS>']
['<BOS>', 'repeated', 'the', 'Portuguese', '<EOS>']
['<BOS>', 'All', 'was', 'joy', 'and', 'gaiety', 'on', 'the', 'ship', 'until', 'long', 'after', 'midnight', '<EOS>']
['<BOS>', 'And', 'so', 'they', 'did', ',', 'all', 'three', ',', 'while', 'the', 'fresh', 'stream', 'gushed', 'forth', 'from', 'its', 'mouth', '<EOS>']


Cool! Seems like the sentences often follow closely with sentences from the book. This is expected as I do no smoothing at all. 

I would like the sentences to derail a little bit more. This could possibly be solved by increasing the size of my training data.

In [294]:
treegram = NGramsTree(5)
treegram.fit(sentences)

C:\Users\Gustav\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [295]:
for i in range(5):
    print(treegram.generate_sentence(sentence = ['<BOS>']))

['<BOS>', '``', 'You', 'bad', 'boy', '<EOS>']
['<BOS>', 'The', 'metallic', 'groups', 'of', 'figures', ',', 'among', 'which', 'were', '``', 'Perseus', "''", 'and', '``', 'The', 'Rape', 'of', 'the', 'Sabines', ',', "''", 'looked', 'like', 'living', 'persons', ',', 'and', 'cries', 'of', 'terror', 'sounded', 'from', 'them', 'all', 'across', 'the', 'noble', 'square', '<EOS>']
['<BOS>', '``', 'Why', ',', 'she', 'will', 'kiss', 'me', ',', 'and', 'say', ',', "'What", 'the', 'goodman', 'does', 'is', 'always', 'right', '.', '<EOS>']
['<BOS>', 'The', 'ant-queen', 'remarked', 'that', 'their', 'conduct', 'that', 'day', 'showed', 'that', 'they', 'possessed', 'kind', 'hearts', 'and', 'good', 'understanding', '<EOS>']
['<BOS>', '``', 'All', 'at', 'once', 'I', 'saw', 'the', 'most', 'charming', 'little', 'people', 'marching', 'towards', 'me', '<EOS>']


With 5-grams the sentences seem to be stolen straight out of the text.